In [20]:
import numpy as np

def LHT(GRAN, op, r):
    lht = {}
    for each in Reps:
        lht[each] = 0
    for l in Locks[GRAN]:
        if op in Locks[GRAN][l]:
            val = service_time[op] + lock_cost[modes[GRAN][MODE][op][l]][place[GRAN][PLACE][l]][r]
            lht[place[GRAN][PLACE][l]] = max(lht[place[GRAN][PLACE][l]], val)
    res = 0
    for each in lht:
        res += lht[each]
    return res
            
    
def op_serialization(GRAN,r):
    res = 0
    for op in freq[WL]:
        exclusive = False
        for l in Locks[GRAN]:
            if op in modes[GRAN][MODE] and l in modes[GRAN][MODE][op] and modes[GRAN][MODE][op][l] == 'X':
                exclusive = True
        if exclusive:
            for r_1 in Reps:
                if r_1 != r:
                    res += LHT(GRAN, op, r_1) * freq[WL][op][r_1]
    return res


def rep_serialization(GRAN,r):
    res = 0
    for op in freq[WL]:
        res += LHT(GRAN, op, r) * freq[WL][op][r]
    return res


def op_parallelism(GRAN,r):
    res = 0
    for r_1 in Reps:
        if r != r_1:
            s = 0
            for op in freq[WL]:
                shared = False
                for l in Locks[GRAN]:
                    if op in modes[GRAN][MODE] and l in modes[GRAN][MODE][op] and modes[GRAN][MODE][op][l] == 'S':
                        shared = True
                if shared:
                    s += LHT(GRAN, op, r_1) * freq[WL][op][r_1]
            res = max(res, s)
    return res


def nonparallelism(GRAN):
    res = 0
    for r in Reps:
        np = abs(op_serialization(GRAN, r) + rep_serialization(GRAN, r) - op_parallelism(GRAN, r))
        res = max(res, np)
    return res


# def coordination_parallelism():
#     res = 0
#     for l in Locks:
#         np = nonparallelism(l)
#         res = max(res, np)
#     return res

def amdahl(GRAN):
    res = 0
    for r in Reps:
        np = op_serialization(GRAN, r) + rep_serialization(GRAN, r) + op_parallelism(GRAN, r)
        res += np
#     res = max(res, np)
    return res/len(Reps)



In [21]:
Reps = ['houston', 'paris', 'singapore']
Ops = ['insert', 'read', 'update']
service_time = {'insert':0, 'read':0, 'update':0}
lock_cost = {'X': {'houston':{'houston':3.2954730850000002, 'paris':223.78096148900002, 'singapore':503.93539986400003}, 
                     'paris':{'houston':223.99058557599997, 'paris':3.2398566490000005, 'singapore':303.88539813499995}, 
                     'singapore':{'houston':503.86683250899995, 'paris':303.965657338, 'singapore':3.108215382}},
             'S': {'houston':{'houston':3.0035555910000005, 'paris':223.53921476500003, 'singapore':503.69265980899996}, 
                     'paris':{'houston':223.67417935400002, 'paris':3.0147095129999997, 'singapore':303.628741349}, 
                     'singapore':{'houston':503.62028737099996, 'paris':303.671423894, 'singapore':2.9168805460000002}}
            }
freq = {'workloadeqeq':{'insert':{'houston':111, 'paris':111, 'singapore':111}, 
                        'read':{'houston':111, 'paris':111, 'singapore':111},
                       'update':{'houston':111, 'paris':111, 'singapore':111}},
       'workloadeqhot':{'insert':{'houston':0, 'paris':333, 'singapore':0}, 
                        'read':{'houston':0, 'paris':333, 'singapore':0},
                       'update':{'houston':0, 'paris':333, 'singapore':0}},
        'workloadeqclust':{'insert':{'houston':167, 'paris':167, 'singapore':0}, 
                           'read':{'houston':167, 'paris':167, 'singapore':0},
                          'update':{'houston':167, 'paris':167, 'singapore':0}},
        'workloadabceq':{'insert':{'houston':303, 'paris':303, 'singapore':303}, 
                     'read':{'houston':25, 'paris':25, 'singapore':25},
                    'update':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadabchot':{'insert':{'houston':0, 'paris':909, 'singapore':0}, 
                      'read':{'houston':0, 'paris':75, 'singapore':0},
                     'update':{'houston':0, 'paris':15, 'singapore':0}},
        'workloadabcclust':{'insert':{'houston':455, 'paris':455, 'singapore':0}, 
                    'read':{'houston':37, 'paris':37, 'singapore':0},
                   'update':{'houston':8, 'paris':8, 'singapore':0}},
        'workloadbaceq':{'insert':{'houston':25, 'paris':25, 'singapore':25}, 
                     'read':{'houston':303, 'paris':303, 'singapore':303},
                    'update':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadbachot':{'insert':{'houston':0, 'paris':75, 'singapore':0}, 
                      'read':{'houston':0, 'paris':909, 'singapore':0},
                     'update':{'houston':0, 'paris':15, 'singapore':0}},
        'workloadbacclust':{'insert':{'houston':37, 'paris':37, 'singapore':0}, 
                    'read':{'houston':455, 'paris':455, 'singapore':0},
                   'update':{'houston':8, 'paris':8, 'singapore':0}}
       }


Locks = {1:{'a-b':{'insert', 'read'}, 'b-c':{'read', 'update'}},
         2:{'a-b-c':{'insert', 'read', 'update'}}}

modes = {1:{1:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'X'}}, 
             2:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'S'}},
             3:{'insert':{'a-b':'X'}, 'read':{'a-b':'X', 'b-c':'S'}, 'update':{'b-c':'X'}},
             4:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'X'}, 'update':{'b-c':'X'}},
             5:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'X'}, 'update':{'b-c':'S'}},
             6:{'insert':{'a-b':'X'}, 'read':{'a-b':'S', 'b-c':'S'}, 'update':{'b-c':'X'}},
             7:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'X'}},
             8:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'X'}, 'update':{'b-c':'S'}},
             9:{'insert':{'a-b':'S'}, 'read':{'a-b':'X', 'b-c':'S'}, 'update':{'b-c':'X'}}},
         2:{1:{'insert':{'a-b-c':'X'}, 'read':{'a-b-c':'X'}, 'update':{'a-b-c':'X'}},
             2:{'insert':{'a-b-c':'X'}, 'read':{'a-b-c':'S'}, 'update':{'a-b-c':'X'}},
             3:{'insert':{'a-b-c':'S'}, 'read':{'a-b-c':'X'}, 'update':{'a-b-c':'S'}}}
        }
place = {1:{1:{'a-b':'houston', 'b-c':'houston'},
             2:{'a-b':'paris', 'b-c':'houston'},
             3:{'a-b':'singapore', 'b-c':'houston'},
             4:{'a-b':'houston', 'b-c':'paris'},
             5:{'a-b':'paris', 'b-c':'paris'},
             6:{'a-b':'singapore', 'b-c':'paris'},
             7:{'a-b':'houston', 'b-c':'singapore'},
             8:{'a-b':'paris', 'b-c':'singapore'},
             9:{'a-b':'singapore', 'b-c':'singapore'}},
         2:{1:{'a-b-c':'houston'},
           2:{'a-b-c':'paris'},
           3:{'a-b-c':'singapore'}}
        }


In [22]:
predictions = {}
for WL in freq:
    predictions[WL] = {}
    for GRAN in Locks:
        for MODE in modes[GRAN]:
            for PLACE in place[GRAN]:
                predictions[WL][str(GRAN) + '-' + str(MODE) + '-' + str(PLACE)] = nonparallelism(GRAN)
#             print(str(MODE) + '-' + str(i+1), '\t', coordination_parallelism())
#             print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', coordination_parallelism())
#             print(coordination_parallelism())
print(predictions)
       


{'workloadeqeq': {'1-1-1': 243426.94086785405, '1-1-2': 280192.343805156, '1-1-3': 342313.463806074, '1-1-4': 280192.34380515607, '1-1-5': 176861.57483988, '1-1-6': 297936.55312075804, '1-1-7': 342313.463806074, '1-1-8': 297936.55312075804, '1-1-9': 270043.254841257, '1-2-1': 193381.65964512003, '1-2-2': 230147.062582422, '1-2-3': 292268.18258334, '1-2-4': 230113.44190664098, '1-2-5': 126782.67294136499, '1-2-6': 247857.651222243, '1-2-7': 274493.369371602, '1-2-8': 230116.45868628597, '1-2-9': 202223.160406785, '1-3-1': 218587.25414257502, '1-3-2': 254909.81326131598, '1-3-3': 285964.66985267104, '1-3-4': 254910.111058443, '1-3-5': 151998.61984094398, '1-3-6': 263773.131074313, '1-3-7': 285964.54980606004, '1-3-8': 263788.13639108697, '1-3-9': 236303.066876739, '1-4-1': 218587.25414257502, '1-4-2': 254910.11105844306, '1-4-3': 285964.54980606004, '1-4-4': 254909.81326131598, '1-4-5': 151998.61984094398, '1-4-6': 263788.13639108697, '1-4-7': 285964.66985267104, '1-4-8': 263773.13107431

In [23]:
amd = {}
for WL in freq:
    amd[WL] = {}
    for GRAN in Locks:
        for MODE in modes[GRAN]:
            for PLACE in place[GRAN]:
                amd[WL][str(GRAN) + '-' + str(MODE) + '-' + str(PLACE)] = amdahl(GRAN)
print(amd)

{'workloadeqeq': {'1-1-1': 243426.94086785402, '1-1-2': 280192.343805156, '1-1-3': 342313.463806074, '1-1-4': 280192.343805156, '1-1-5': 176861.57483988, '1-1-6': 297936.55312075804, '1-1-7': 342313.463806074, '1-1-8': 297936.553120758, '1-1-9': 270043.254841257, '1-2-1': 234847.54593351204, '1-2-2': 271612.948870814, '1-2-3': 333734.068871732, '1-2-4': 271604.709339112, '1-2-5': 168273.94037383597, '1-2-6': 289348.918654714, '1-2-7': 330780.507372194, '1-2-8': 286403.59668687795, '1-2-9': 258510.298407377, '1-3-1': 288998.05603288306, '1-3-2': 348265.72244188795, '1-3-3': 400016.847827093, '1-3-4': 348261.3660691771, '1-3-5': 207636.745968182, '1-3-6': 363044.260246123, '1-3-7': 400019.55887188995, '1-3-8': 363054.21050952905, '1-3-9': 318576.12976842903, '1-4-1': 288998.05603288306, '1-4-2': 348261.366069177, '1-4-3': 400019.55887188995, '1-4-4': 348265.722441888, '1-4-5': 207636.745968182, '1-4-6': 363054.21050952905, '1-4-7': 400016.847827093, '1-4-8': 363044.26024612295, '1-4-9': 